In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
train = pd.read_csv("train.csv", parse_dates=['Datetime'])
test = pd.read_csv("test.csv", parse_dates=['Datetime'])

In [4]:
num_train = len(train)
y_train1=train['Price']
y_train2=train['Number_Of_Sales']
train=train.drop('Price',axis=1)
train=train.drop('Number_Of_Sales',axis=1)

In [5]:
df_all = pd.concat([train, test])

In [7]:
total=df_all
from sklearn.preprocessing import LabelEncoder
en=LabelEncoder()
total['Datetime']=en.fit_transform(total['Datetime'])

In [8]:
data_to_pred=df_all.loc[total['Category_2'].isnull()]

In [9]:
test_df=total.dropna(axis=0, how='any')

In [10]:
print (test_df.shape)
target=test_df['Category_2']
test_df=test_df.drop(['Category_2'],axis=1)
test_df=test_df.drop(['ID'],axis=1)


(1017319, 6)


In [11]:
from sklearn.cross_validation import train_test_split
X_tr,X_te,y_tr,y_te=train_test_split(test_df,target,test_size=0.2,random_state=400)


/home/seshaditya/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


import xgboost as xgb
xgb_params = {
    'n_trees': 1000, 
    'eta': 0.05,
    'max_depth':14,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtr = xgb.DMatrix(X_tr,y_tr)
dte = xgb.DMatrix(X_te)

num_boost_rounds = 1000
model = xgb.train(dict(xgb_params, silent=0), dtr, num_boost_round=num_boost_rounds)


pred = model.predict(dte)
#pred2=pred2.clip(0)
#pred2=np.around(pred2, decimals=0)
r2=r2_score(y_te,pred)
mse=mean_squared_error(y_te,pred)

print("mean square error and r2 score of xgb is {},{}".format(r2,mse))

mean square error and r2 score of xgb is 0.99850092368,0.00239961005962


In [13]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


import xgboost as xgb
xgb_params = {
    'n_trees': 1000, 
    'eta': 0.05,
    'max_depth':14,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtr = xgb.DMatrix(test_df,target)


num_boost_rounds = 1000
model = xgb.train(dict(xgb_params, silent=0), dtr, num_boost_round=num_boost_rounds)




In [14]:
data_to_pred.head(n=5)

,Category_1,Category_2,Category_3,Datetime,ID,Item_ID
43,0,NaN,0,0,30653_20140101,30653
44,0,NaN,0,0,29955_20140101,29955
207,0,NaN,0,0,30147_20140101,30147
358,0,NaN,0,0,30347_20140101,30347
359,0,NaN,0,0,29662_20140101,29662


In [15]:
data_to_pred=data_to_pred.drop(['ID'],axis=1)

data_to_pred=data_to_pred.drop(['Category_2'],axis=1)

In [16]:
dte = xgb.DMatrix(data_to_pred)

In [17]:
p=model.predict(dte)

In [18]:
print len(p)

130805


In [19]:
df_all['Category_2'].loc[df_all['Category_2'].isnull()]=p

/home/seshaditya/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
df_all.to_csv('cleaned_df_all_new.csv', index=False)